In [ ]:
# Importar OpenCV
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Definir função 'dummy'
def passFunction(x):
    pass

In [ ]:
# Criar nome da janela
window_title = 'Tilted image'
cv.namedWindow(window_title)

# Importar imagem e criar duas imagens a partir da imagem de base: sem borramento e com borramento
image = cv.imread('handwriting.jpg', cv.IMREAD_COLOR)

image1 = image.copy()
image2 = cv.blur(image, (5,5))

In [ ]:
# Creação das trackbars

d_val_max = 100
trackbar_d = 'Suavidade'
cv.createTrackbar(trackbar_d, window_title , 0, d_val_max, passFunction)

trackbar_L = 'Tamanho'
cv.createTrackbar(trackbar_L, window_title, 0, image.shape[0], passFunction)

trackbar_C = 'Centro'
cv.createTrackbar(trackbar_C, window_title, int(image.shape[0]/2), int(image.shape[0]), passFunction)

In [ ]:
# Criar variável para inicializar janela pós processada
first_pass = 1

In [ ]:
# Criar função principal
while(True):
    
    # Exibir imagem sem borramento se for a primeira passagem do programa
    if first_pass == 1:
        cv.imshow(window_title,image)
    else:
        # Exibir imagem com borramento após a primeira passagem
        cv.imshow(window_title,tilted)

    # Caso seja pressionado o botão ESC sair do programa
    k = cv.waitKey(1) & 0xFF
    if k == 27:
        break

    # L é o tamanho da parte borrada da imagem
    L = cv.getTrackbarPos(trackbar_L,window_title)
    
    # Definição de L1 e L2 que dizem respeito à linha onde o borramento termina com relação à linha 0
    L1 = L/2
    L1 = max(0,L1)
    L2 = image.shape[0]-L1

    # Definição de d como suavidade do borramento
    d = cv.getTrackbarPos(trackbar_d,window_title)
    
    # Ajuste do valor mínimo de d para não dividir por zero
    d = 0.01+d
    
    # Definição do centro do foco, começando na linha 0 e indo até a última linha
    C = cv.getTrackbarPos(trackbar_C,window_title)
    C = C - image.shape[0]/2
    if C < 0:
        C = max(C,-L1)
    else:
        C = min(C,L1)

    # Criação da função alpha(x)
    x = np.arange(0,image1.shape[0],1)
    alpha = (np.tanh((x-L1-C)/d) - np.tanh((x-L2-C)/d))/2

    # Plot da função alpha(x)
    _ = plt.plot(x,alpha)
    plt.show()

    # Ajuste necessário para operar sobre a matriz da imagem
    alpha = alpha.reshape(-1,1,1)
    alpha_mat = np.tile(alpha,(1,image1.shape[1],image1.shape[2]))
    beta_mat = 1-alpha_mat
    
    # Criação da imagem de saída a partir de alpha e das imagens de entrada (com e sem borramento)
    tilted = np.multiply(alpha_mat,image1) + np.multiply(beta_mat,image2)
    tilted = tilted.astype(np.uint8)

    # Modificar a variável após primeira passagem
    first_pass = 0

# Fechar as janelas ao sair do programa
cv.destroyAllWindows()